In [2]:
import numpy as np
import pandas as pd
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# DATOS Y CARGA

In [3]:
# Cargar los datos directamente desde fetch_openml
mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)

# Obtener los datos y las etiquetas
X = np.array(mnist.data)
y = np.array(mnist.target)
X.shape,y.shape

((70000, 784), (70000,))

# NORMALIZACIÓN EXTENDIDA

In [4]:
X_norm=normalize(np.append(X,np.ones((X.shape[0],1),dtype=float).reshape(-1,1),axis=1))
X_norm.shape,type(X_norm)

((70000, 785), numpy.ndarray)

### REESCALADO DE LOS DATOS. 600 PARA APRENDIZAJE Y 100 PARA TEST

In [5]:
XR=X_norm[:700]
YR=y[:700]
XR.shape,YR.shape

((700, 785), (700,))

# CLASES A UTILIZAR PARA SOM

### Clase Neurona

In [6]:
class neurona(object):
    def __init__(self, f=0, c=0, dim=0):
        self.c = c
        self.f = f
        self.dim = dim
        self.w = normalize(0.5 - np.random.rand(self.dim).reshape(1,-1))
        self.output = -10.0
        self.label = -1
            
    def predict(self, input):
        self.output = input @ self.w.T
        return self.output
    
    def fit(self, input, alfa=1):
        self.w = normalize(self.w + (alfa*input))
            
    def labeling(self, inputs, targets):
        Y = inputs @ self.w.T
        self.label = targets[np.argmax(Y)]

            

### Clase SOM

In [7]:
class som():
    
    def __init__(self, filas=1, columnas=1, dim=1, init_radious=0, init_alfa=1):
        self.neronas = []
        self.filas = filas
        self.columnas = columnas
        self.dim = dim
        # self.labels = []
        self.radious = init_radious
        self.init_alfa = init_alfa
        self.t = 0
        # Considera que un mapa rectangular es una lista de objetos "neurona", que viene localizado por sus atributos "fila" y "columna"
        for fila in range(self.filas):
            for columna in range(self.columnas):
                self.neronas.append(neurona(f=fila, c=columna, dim=self.dim))
                
    def fit(self, inputs, max_epochs=1):
        # método similar a otros algoritmo de ML. Recibe las entradas, el radio inicial, el factor de apendizaje inicial,
        # el máximo de épocas y devuelve los pesos ajustados
        P = inputs.shape[0]
        for epoch in range(max_epochs):
            for x in inputs:
                self.alfa = self.init_alfa/(1.0 + float(self.t/P))
                i_gana, y_gana = -1, float('-inf')
                for i in range(self.filas*self.columnas):
                    y_predict = self.neronas[i].predict(x.reshape(1,-1))
                    if y_predict > y_gana:
                        y_gana = y_predict
                        i_gana = i
                f_gana = int(i_gana / self.columnas)
                c_gana = i_gana % self.columnas
                
                # Conjunto de vecinas para un radious
                for f in range(f_gana - self.radious, f_gana + self.radious+1):
                    if f < 0:
                        row = self.filas + f
                    else:
                        if f > self.filas-1:
                            row = f % self.filas
                        else:
                            row = f

                    for c in range(c_gana - self.radious, c_gana + self.radious+1):
                        if c < 0:
                            column = self.columnas + c 
                        else:
                            if c > self.columnas-1:
                                column = c % self.columnas
                            else:
                                column = c
                        self.neronas[(row*self.columnas) + column].fit(x.reshape(1,-1), alfa=self.alfa)
                self.t += 1
                if (self.t % 1000) == 0:
                    print("t =", self.t, "  R =", self.radious, "  epoch =", epoch, "\t\t", end='')
            if self.radious > 0:
                self.radious -= 1
                            
    def neuron_labeling(self, inputs, targets):
        # recorre la neronas de neuronas y va llamanado a su metodo de etiquetado para cada neurona
        for i in range(self.filas*self.columnas):
            self.neronas[i].labeling(inputs, targets)

    def predict(self, inputs):
        # recorre la lista de neuronas y calcula la salida de un conjunto de muestras
        # util para usar la salida del som como entrada a otrso sistemas
        output_list = []
        for x in inputs:
            for i in range(self.filas*self.columnas):
                output_list.append(self.neronas[i].predict(x.reshape(1,-1)))
        return np.array(output_list).reshape(inputs.shape[0], -1)    
    
    def label_predict(self, inputs):
        # clasificación de muestras con el etiquetado de cada neurona hecho previamente
        label_list = []
        i_gana = -1
        for x in inputs:
            i_gana = np.argmax(np.array([self.neronas[i].predict(x) for i in range(self.filas*self.columnas)]))
            label_list.append(self.neronas[i_gana].label)
        return np.array(label_list).reshape(inputs.shape[0], )

# Inicialización del SOM. 

In [8]:
X_train,X_test,y_train,y_test=train_test_split(XR,YR,test_size=100,stratify=YR)

In [9]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((600, 785), (100, 785), (600,), (100,))

In [10]:
mapa_autorganizado=som(filas=15,columnas=9,dim=X_train.shape[1],init_radious=4,init_alfa=10)

In [11]:
mapa_autorganizado.fit(inputs=X_train,max_epochs=20)

t = 1000   R = 3   epoch = 1 		t = 2000   R = 1   epoch = 3 		t = 3000   R = 0   epoch = 4 		t = 4000   R = 0   epoch = 6 		t = 5000   R = 0   epoch = 8 		t = 6000   R = 0   epoch = 9 		t = 7000   R = 0   epoch = 11 		t = 8000   R = 0   epoch = 13 		t = 9000   R = 0   epoch = 14 		t = 10000   R = 0   epoch = 16 		t = 11000   R = 0   epoch = 18 		t = 12000   R = 0   epoch = 19 		

In [12]:
mapa_autorganizado.neuron_labeling(X_train,y_train)

In [13]:
etiquetas_neuronas_predict=mapa_autorganizado.label_predict(X_test)

In [14]:
accuracy_score(y_test,etiquetas_neuronas_predict)

0.72

# MLP (135X60X10)

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [16]:
X_train_mlp=mapa_autorganizado.predict(X_train)

In [17]:
X_train_mlp.shape

(600, 135)

In [18]:
X_test_mlp=mapa_autorganizado.predict(X_test)

In [19]:
X_test_mlp.shape

(100, 135)

### ESCALADO

In [20]:
escalado=MinMaxScaler()

In [21]:
escalado.fit(np.vstack((X_train_mlp,X_test_mlp)))

MinMaxScaler()

In [22]:
X_train_mlp=escalado.transform(X_train_mlp)

In [23]:
X_test_mlp=escalado.transform(X_test_mlp)

In [24]:
from sklearn.neural_network import MLPClassifier

In [25]:
mlp=MLPClassifier(hidden_layer_sizes=(60,),verbose=True,max_iter=3000) 

In [26]:
mlp.fit(X_train_mlp,y_train)

Iteration 1, loss = 2.35019160
Iteration 2, loss = 2.24487865
Iteration 3, loss = 2.18358167
Iteration 4, loss = 2.13913795
Iteration 5, loss = 2.09585030
Iteration 6, loss = 2.05711872
Iteration 7, loss = 2.01747285
Iteration 8, loss = 1.97713412
Iteration 9, loss = 1.93957204
Iteration 10, loss = 1.90368999
Iteration 11, loss = 1.86810911
Iteration 12, loss = 1.83496612
Iteration 13, loss = 1.80060374
Iteration 14, loss = 1.76722113
Iteration 15, loss = 1.73588438
Iteration 16, loss = 1.70504574
Iteration 17, loss = 1.67438028
Iteration 18, loss = 1.64571149
Iteration 19, loss = 1.61653959
Iteration 20, loss = 1.58991836
Iteration 21, loss = 1.56141282
Iteration 22, loss = 1.53636315
Iteration 23, loss = 1.51079658
Iteration 24, loss = 1.48711515
Iteration 25, loss = 1.46510660
Iteration 26, loss = 1.44264801
Iteration 27, loss = 1.42258282
Iteration 28, loss = 1.40157670
Iteration 29, loss = 1.38270949
Iteration 30, loss = 1.36521269
Iteration 31, loss = 1.34677885
Iteration 32, los

Iteration 265, loss = 0.65818308
Iteration 266, loss = 0.65892514
Iteration 267, loss = 0.65976363
Iteration 268, loss = 0.65939818
Iteration 269, loss = 0.65570676
Iteration 270, loss = 0.65554130
Iteration 271, loss = 0.65400422
Iteration 272, loss = 0.65297762
Iteration 273, loss = 0.65608407
Iteration 274, loss = 0.65026547
Iteration 275, loss = 0.65350327
Iteration 276, loss = 0.65220481
Iteration 277, loss = 0.65022108
Iteration 278, loss = 0.64860182
Iteration 279, loss = 0.65597164
Iteration 280, loss = 0.64908525
Iteration 281, loss = 0.65114001
Iteration 282, loss = 0.64909424
Iteration 283, loss = 0.64628087
Iteration 284, loss = 0.64731779
Iteration 285, loss = 0.64680969
Iteration 286, loss = 0.64372148
Iteration 287, loss = 0.64505593
Iteration 288, loss = 0.64373154
Iteration 289, loss = 0.64336236
Iteration 290, loss = 0.64119384
Iteration 291, loss = 0.64231746
Iteration 292, loss = 0.64064518
Iteration 293, loss = 0.63949757
Iteration 294, loss = 0.63831284
Iteration 

Iteration 553, loss = 0.52339836
Iteration 554, loss = 0.52382921
Iteration 555, loss = 0.52163875
Iteration 556, loss = 0.52321839
Iteration 557, loss = 0.52278891
Iteration 558, loss = 0.51978479
Iteration 559, loss = 0.52127641
Iteration 560, loss = 0.51730988
Iteration 561, loss = 0.52106694
Iteration 562, loss = 0.51855137
Iteration 563, loss = 0.52093925
Iteration 564, loss = 0.52288744
Iteration 565, loss = 0.51905205
Iteration 566, loss = 0.51782066
Iteration 567, loss = 0.51838507
Iteration 568, loss = 0.51633650
Iteration 569, loss = 0.51782051
Iteration 570, loss = 0.51859510
Iteration 571, loss = 0.51661090
Iteration 572, loss = 0.51600756
Iteration 573, loss = 0.51560409
Iteration 574, loss = 0.51508173
Iteration 575, loss = 0.51731543
Iteration 576, loss = 0.51368894
Iteration 577, loss = 0.51590827
Iteration 578, loss = 0.51560980
Iteration 579, loss = 0.51722055
Iteration 580, loss = 0.51522859
Iteration 581, loss = 0.51512219
Iteration 582, loss = 0.51434197
Iteration 

Iteration 843, loss = 0.45553180
Iteration 844, loss = 0.45612189
Iteration 845, loss = 0.45227297
Iteration 846, loss = 0.45124455
Iteration 847, loss = 0.45258014
Iteration 848, loss = 0.45413516
Iteration 849, loss = 0.45159947
Iteration 850, loss = 0.45159372
Iteration 851, loss = 0.44982626
Iteration 852, loss = 0.45205079
Iteration 853, loss = 0.45550421
Iteration 854, loss = 0.45064835
Iteration 855, loss = 0.45324065
Iteration 856, loss = 0.44974402
Iteration 857, loss = 0.45054430
Iteration 858, loss = 0.45222759
Iteration 859, loss = 0.45277597
Iteration 860, loss = 0.44943178
Iteration 861, loss = 0.45121814
Iteration 862, loss = 0.45015586
Iteration 863, loss = 0.45165593
Iteration 864, loss = 0.44847265
Iteration 865, loss = 0.44811612
Iteration 866, loss = 0.44957985
Iteration 867, loss = 0.44711073
Iteration 868, loss = 0.44761360
Iteration 869, loss = 0.44795058
Iteration 870, loss = 0.44582158
Iteration 871, loss = 0.44677298
Iteration 872, loss = 0.45001416
Iteration 

MLPClassifier(hidden_layer_sizes=(60,), max_iter=3000, verbose=True)

In [27]:
mlp.predict(X_test_mlp)

array(['7', '5', '4', '2', '4', '7', '8', '8', '1', '6', '4', '5', '1',
       '5', '1', '3', '5', '2', '6', '9', '5', '9', '7', '9', '9', '8',
       '0', '8', '4', '8', '4', '2', '0', '7', '6', '4', '8', '1', '3',
       '2', '8', '9', '5', '2', '7', '4', '0', '2', '6', '8', '6', '4',
       '5', '9', '1', '3', '6', '9', '8', '1', '9', '7', '0', '9', '7',
       '0', '2', '6', '4', '3', '8', '3', '2', '4', '8', '0', '5', '8',
       '7', '6', '1', '4', '4', '3', '4', '3', '3', '1', '9', '6', '0',
       '7', '2', '1', '7', '9', '6', '9', '0', '3'], dtype='<U1')

In [28]:
accuracy_score(y_test,mlp.predict(X_test_mlp))

0.82